In [1]:
import pandas as pd

dataset_id = "1KfvUPTtutsF63NxNXLtoFJQRASm1WwP3" #dataset-cleaned.csv
url = f"https://drive.google.com/uc?id={dataset_id}"

df = pd.read_csv(url)

In [2]:
!pip install transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 100.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjit

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-14B")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-14B", device_map="auto", torch_dtype=torch.float16)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/9.73k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

model-00007-of-00008.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/3.84G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [4]:
from transformers import pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

Device set to use cuda:0


In [5]:
import re
def translate(sentence):
    prompt = (
        f'You are an expert at translating text from non modern Italian to modern Italian. Non modern Italian is a form of the Italian language that was directly derived from Ancient Latin. Provide the translation of the following sentence written between 1250 and 1350, specifically the Tuscan and Umbrian variety, into modern Italian.\n'
        f'Only return the translated sentence enclosed in double quotes. No notes, no explanations.\n\n'
        f'Old Italian: "{sentence}"\n'
        f'Modern Italian:'
    )

    output = pipe(prompt, max_new_tokens=100, do_sample=False)[0]["generated_text"]

    # Extract only what follows "Modern Italian:"
    response = output.split("Modern Italian:")[-1].strip()

    # Extract only what is inside the quotation marks ""
    match = re.search(r'"([^"]+)"', response)

    if match:
        translation = match.group(1).strip()
    else:
        # Fallback: use entire response, clean newlines
        translation = response.strip().split('\n')[0]

    return translation


In [6]:
df['Translation'] = df['Sentence'].apply(translate)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p'

In [11]:
df

,Author,Date,Region,Sentence,Translation
0,Brunetto Latini,1260-61,fior.,quella guerra ben fatta l' opera perché etc. E...,quella guerra fu ben fatta l' opera perché etc...
1,Bono Giamboni,1292,fior.,"crudele, e di tutte le colpe pigli vendetta, c...","Crudele, e prendi vendetta per tutte le colpe,..."
2,Valerio Massimo (red. V1,1336,fior.,Non d' altra forza d' animo fue ornato Ponzio ...,"Ponzio Aufidiano, cavaliere romano, non era or..."
3,Lucano volg. (ed. Marinoni),1330/40,prat.,Se questo piace a tutti e se 'l tempo hae biso...,Se a tutti piace e se il tempo ha bisogno di a...
4,Brunetto Latini,1260-61,fior.,Officio di questa arte pare che sia dicere app...,"""E quindi, quando si parla dell'arte, si parla..."
...,...,...,...,...,...
92,Grazia di Meo,1343,tosc.,"Alchuno è riccho e gentile, ma lamentasi che v...","Qualcuno è ricco e gentile, ma si lamenta che ..."
93,Zanobi da Strada,1361,tosc.,"Pietro, essendogli mostrato in figura il popul...","Pietro, dopo che gli fu mostrato in figura il ..."
94,Declamazioni di Seneca,1392,tosc.,pregollo che lo liberasse di quella obbligazio...,"pregò che lo liberasse da quell'obbligazione, ..."
95,Giovanni da San Miniato,1415,tosc.,L'oro verrà dall'Aquilone. Che figuriamo noi p...,L'oro verrà dall'Aquilone. Che figuriamo noi p...


In [12]:
# Salva in formato JSONL
df.to_json('giuminia-hw2_transl-qwen3-14B-2.jsonl', orient='records', lines=True, force_ascii=False)

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
import shutil

source_path = '/content/giuminia-hw2_transl-qwen3-14B-2.jsonl'
destination_path = '/content/drive/MyDrive/model_jsonl/giuminia-hw2_transl-qwen3-14B-2.jsonl'


shutil.copy(source_path, destination_path)

'/content/drive/MyDrive/model_jsonl/giuminia-hw2_transl-qwen3-14B-2.jsonl'